<a href="https://colab.research.google.com/github/rishitha957/Sequence-Alignments/blob/main/cgseq_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import random

In [2]:
def scores():
    match = 1
    mismatch = -1
    gap_penalty = -1
    return match, mismatch, gap_penalty

In [3]:
def needleman_wunsch(seq1, seq2):
    match, mismatch, gap_penalty = scores()
    rows = len(seq1) + 1
    cols = len(seq2) + 1
    dp_matrix = [[0] * cols for _ in range(rows)]

    for i in range(1, rows):
        dp_matrix[i][0] = i * gap_penalty
    for j in range(1, cols):
        dp_matrix[0][j] = j * gap_penalty

    for i in range(1, rows):
        for j in range(1, cols):
            match_score = dp_matrix[i - 1][j - 1] + (match if seq1[i - 1] == seq2[j - 1] else mismatch)
            delete_score = dp_matrix[i - 1][j] + gap_penalty
            insert_score = dp_matrix[i][j - 1] + gap_penalty
            dp_matrix[i][j] = max(match_score, delete_score, insert_score)

    return dp_matrix

In [4]:
def find_alignment(seq1, seq2, score_matrix):
    match, mismatch, gap_penalty = scores()
    rows = len(seq1) + 1
    cols = len(seq2) + 1
    alignment1 = []
    alignment2 = []
    i, j = rows - 1, cols - 1
    while i > 0 or j > 0:
        current_score = score_matrix[i][j]
        diagonal_score = score_matrix[i - 1][j - 1] if i > 0 and j > 0 else float('-inf')
        up_score = score_matrix[i - 1][j] if i > 0 else float('-inf')
        left_score = score_matrix[i][j - 1] if j > 0 else float('-inf')

        if i > 0 and j > 0 and current_score == diagonal_score + (match if seq1[i - 1] == seq2[j - 1] else mismatch):
            alignment1.insert(0, seq1[i - 1])
            alignment2.insert(0, seq2[j - 1])
            i -= 1
            j -= 1
        elif i > 0 and current_score == up_score + gap_penalty:
            alignment1.insert(0, seq1[i - 1])
            alignment2.insert(0, '-')
            i -= 1
        elif j > 0:
            alignment1.insert(0, '-')
            alignment2.insert(0, seq2[j - 1])
            j -= 1

    aligned_seq1 = ''.join(alignment1)
    aligned_seq2 = ''.join(alignment2)

    return aligned_seq1, aligned_seq2


In [5]:
seq1 = ''.join(random.choice(['a','c','g','t']) for _ in range(10000))
seq2 = ''.join(random.choice(['a','c','g','t']) for _ in range(10000))
s = time.time()
dp_matrix = needleman_wunsch(seq1, seq2)
t = time.time()
print("Time taken to construct NW DP matrix:",(t-s),"s")
s = time.time()
aligned_seq1, aligned_seq2 = find_alignment(seq1, seq2, dp_matrix)
t = time.time()
print("Time taken to find best alignment:",(t-s)*1000,"ms")
print("Sequence 1:", seq1)
print("Sequence 2:", seq2)
print("Aligned Sequence 1:", aligned_seq1)
print("Aligned Sequence 2:", aligned_seq2)
print("Alignment score:",dp_matrix[len(seq1)-1][len(seq2)-1])

Time taken to construct NW DP matrix: 91.77244329452515 s
Time taken to find best alignment: 85.26349067687988 ms
Sequence 1: ggtaaaacggttctactccatcacagacttggtgtgcgacagcagactcccttgtgccgacgttgtataagctccgggaccacctgatataaaatcactcttcccctgatttattcgaacacgctccgtcctagctcatcaccgctgcctcatttaccccacaaacgtctgcctggatcgtcatacagaaaccaacaagttcccgaagagtattgcgtaacattccggcggaacctttcacttgacatcgccagcattgtgcatctgacgagcgcacaccctccactacggaatgcttgtaggctggtccacccgatatagtcccattctcaccttgtcggttgaggccacttgtccaggatagaacgttcttcgggttgagttatccaagttaagtgcaaaagatcagaagaaaagttatcgtagttaccttaaggcccgttcgaataacaacaacaagatactggtgtccaccaacaccattactctcgcagagacattgtcttctaacaggggcatactgcagaacccggccagcgcttctactcgtgtacaaaattcagtcctgtgggcctacgatattgagcccggtcaccctctatctgcctgtaccccggcgatttggcccgcgcatccggctccatgactcatcattcccggcaacccagtattcccccagcttagccggtagactttccacgggcgatgatggatgggctcctgttgccatcacagcaaccgagaagcacaatgcgacgcgatatccccccttggttaattctagggattaagaagttggagaggcggaccgttacgaccggaggagtggcatcaggactctgtgcaccatggtcagaattcccgggacctatct

Hirsberg's Algorithm

In [6]:
def hirschberg(seq1, seq2):

    def linear_space_score(seq1, seq2):
        match, mismatch, gap_penalty = scores()
        len1 = len(seq1)
        len2 = len(seq2)
        dp = [[0 for j in range(len1 + 1)] for i in range(2)]
        curridx = 0
        for i in range(len2 + 1):
            for j in range(len1 + 1):
                if i == 0 and j == 0:
                    dp[curridx][j] = 0
                elif i > 0 and j == 0:
                    dp[curridx][j] = dp[1 - curridx][j] + gap_penalty
                elif i == 0 and j > 0:
                    dp[curridx][j] = dp[curridx][j - 1] + gap_penalty
                else:
                    score = match if seq1[j - 1] == seq2[i - 1] else mismatch
                    dp[curridx][j] = max(dp[1 - curridx][j] + gap_penalty,
                                         max(dp[curridx][j - 1] + gap_penalty, dp[1 - curridx][j - 1] + score))
            curridx = 1 - curridx
        return dp[1 - curridx]

    def hirschberg_recur(seq1, seq2):
        if len(seq1) == 0:
            return "-" * len(seq2),seq2
        if len(seq2) == 0:
            return seq1,"-" * len(seq1)

        if len(seq1) == 1 or len(seq2) == 1:
            aligned_seq1 = ""
            aligned_seq2 = ""
            dp = linear_space_score(seq2, seq1) if len(seq1) == 1 else linear_space_score(seq1, seq2)
            dp[0] = 0
            match, mismatch, gap_penalty = scores()
            for i in range(len(dp) - 1):
                if dp[i + 1] - dp[i] == gap_penalty:
                    aligned_seq1 += '-' if len(seq1) == 1 else seq1[i]
                    aligned_seq2 += seq2[i] if len(seq1) == 1 else '-'
                else:
                    aligned_seq1 += seq1 if len(seq1) == 1 else seq1[i]
                    aligned_seq2 += seq2[i] if len(seq1) == 1 else seq2
            return aligned_seq1, aligned_seq2

        mid = int(len(seq2) / 2)
        prefix = linear_space_score(seq1, seq2[:mid])
        suffix = linear_space_score(seq1[::-1], seq2[mid:][::-1])[::-1]
        total_scores = [pre + suf for pre, suf in zip(prefix, suffix)]
        maxidx = total_scores.index(max(total_scores))

        pre_seq1, pre_seq2 = hirschberg_recur(seq1[:maxidx], seq2[:mid])
        suf_seq1, suf_seq2 = hirschberg_recur(seq1[maxidx:], seq2[mid:])

        return pre_seq1 + suf_seq1, pre_seq2 + suf_seq2

    aligned_seq1, aligned_seq2 = hirschberg_recur(seq1, seq2)

    return aligned_seq1, aligned_seq2

In [7]:
# seq1 = ''.join(random.choice(['a','c','g','t']) for _ in range(10000))
# seq2 = ''.join(random.choice(['a','c','g','t']) for _ in range(10000))
s = time.time()
aligned_seq1, aligned_seq2 = hirschberg(seq1, seq2)
t = time.time()
print("Time taken to find best alignment:",(t-s),"s")
print("Sequence 1:", aligned_seq1)
print("Sequence 2:", aligned_seq2)

Time taken to find best alignment: 229.26296257972717 s
Sequence 1: g--taaa-acg-g---t-tac----a--tca-cagact---gtgtgcg-c---ca-a--t--ccct-tgt-c-gacgttgt--aagc-tc--cgggacca-c-tgat---ta-a-a--t-act-ttc--ccc-----tga--tta--t-c--g-a-ac--acgc--t----c-----g-tcctagctc-a--t-c--a--gctgcctca-tttac-----ccc----a--ca--c--g-t--c--gcct-gg--at---cg-ca--a-caga-ac---c-aacaagt-----tc-cg-agagta-tgc-ta-ac--attccggcggaacct--ttc-cttg-ac---atcgc-a--cat-t-gt---g-catctgacga-cgcacacc-tcca--t-cgg--t-ctt---g-a-ggctggtcca-ccg-t-t-gtccc--tt-tc-ac--t-gtc-g-tg-agg--a-cttgt--cca-ga-tagaa-gttct--c-gg--g---ga--g-ta-t-ca-g-ttaagtgc--aa-at-a--aa-a--agtt--c-g-ag--tt--ac-ctta-g--ccgt-t-g-a---a-t--caac--a--c---a-g-tac-gg-tgtcc-ccaa-c---ccat--a--tct-g--cagag---ac-a-tgt-ttcta-a-agg--gcatactgcagaacccg-gcca-gcgct-tctactcgtgtacaaaattcagt-ctgtgggc-c-t-cga-attgagcccggtc--c-tct-tc-tgcctg-ac---c-gcga-ttggc-cgcgc--tccg-g-tc-a-gac-c-t-a--t--tcc-cggcaac-cagta----cccagc-ttagccggtagac-ttccacgggc-gatg-at--a-gg-ct-----gttgccatcaca-gcaaccg---aag-a